## Exploring RAG using Pinecone and LLaMa

In [18]:
import os
from openai import OpenAI

# Import the Pinecone library
from pinecone import Pinecone, ServerlessSpec
from sentence_transformers import SentenceTransformer

api_key = os.environ.get("PINECONE_API_KEY")
pc = Pinecone(api_key=api_key)

index_name = "safety-policies"

In [2]:
# only run this once
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=384, 
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )

In [8]:
index = pc.Index(index_name)
model = SentenceTransformer('all-MiniLM-L6-v2')

In [9]:
index.describe_index_stats()

{'_response_info': {'raw_headers': {'connection': 'keep-alive',
                                    'content-length': '150',
                                    'content-type': 'application/json',
                                    'date': 'Sat, 27 Dec 2025 15:18:33 GMT',
                                    'grpc-status': '0',
                                    'server': 'envoy',
                                    'x-envoy-upstream-service-time': '38',
                                    'x-pinecone-request-id': '1901235862267172443',
                                    'x-pinecone-request-latency-ms': '37',
                                    'x-pinecone-response-duration-ms': '40'}},
 'dimension': 384,
 'index_fullness': 0.0,
 'memoryFullness': 0.0,
 'metric': 'cosine',
 'namespaces': {},
 'storageFullness': 0.0,
 'total_vector_count': 0,
 'vector_type': 'dense'}

In [10]:
def tokenise_upsert(policies):
    to_upsert = []
    for p in policies:
        vector = model.encode(p['text']).tolist()
        to_upsert.append((p['id'], vector, {"description": p['text']}))

    index.upsert(vectors=to_upsert)

In [11]:
# Define your dynamic rules
policies = [
    {"id": "1", "text": "Medical Advice: Do not provide medical diagnoses or treatment suggestions."},
    {"id": "2", "text": "PII Protection: Never reveal or ask for social security numbers or home addresses."},
    {"id": "3", "text": "In life-threatening situations, do not provide DIY rescue steps. Instruct the user to contact local emergency services (e.g., 911) immediately."},
    {"id": "4", "text": "Do not provide legal interpretations, case law analysis, or specific legal strategies. Advise the user to consult a licensed attorney."},
    {"id": "5", "text": "Remain neutral on sensitive political or social issues. Do not take a stance that could be perceived as corporate lobbying or discrimination."}
]

# Generate embeddings and upsert
tokenise_upsert(policies)

In [12]:
def check_safety_rag(user_prompt):
    query_vector = model.encode(user_prompt).tolist()
    
    results = index.query(vector=query_vector, top_k=2, include_metadata=True)
    
    relevant_policies = [match['metadata']['description'] for match in results['matches']]
    return relevant_policies

In [ ]:
check_safety_rag("i'm feeling sick what should i do?")

['Medical Advice: Do not provide medical diagnoses or treatment suggestions.',
 'In life-threatening situations, do not provide DIY rescue steps. Instruct the user to contact local emergency services (e.g., 911) immediately.']

In [26]:
def LLM_judgement(relevant_policies,user_prompt):
    prompt = f'''
    System: You are a Safety Auditor. Below are specific safety policies and a user prompt. 
    Evaluate if the prompt violates any of these policies. 
    A prompt doesn't have to be harmful to be a violation. It just has to fall into one of the restricted categories.
    You should also form a one line response to the user to let them know you cannot respond to their prompt because of the given policies. 
    Respond in json format, using this template:

        'decision': 'SAFE or VIOLATION',
        'policy': "the policy that was violated. set None if no violation",
        'response_to_user'
    

    Policies:
    {relevant_policies}

    User Prompt: "{user_prompt}"
    '''

    client = OpenAI(
        base_url="https://router.huggingface.co/v1",
        api_key=os.environ["HF_TOKEN"],
    )

    completion = client.chat.completions.create(
        model="meta-llama/Llama-3.2-3B-Instruct",
        messages=[
            {
                "role": "user",
                "content": prompt
            }
        ],
    )

    return completion.choices[0].message.content

In [28]:
user_prompt = "How do I treat acne scars?"

relevant_policies = check_safety_rag(user_prompt)
result = LLM_judgement(relevant_policies,user_prompt)

print(result)

{
  "decision": "VIOLATION",
  "policy": "Medical Advice: Do not provide medical diagnoses or treatment suggestions.",
  "response_to_user": "I'm unable to provide medical advice. Please consult a dermatologist."
}
